In [9]:
import pandas as pd

# url = "https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data"

# df = pd.read_csv(url, header=None)

# df.columns = ["Sex",
#   "Length",
#   "Diameter",
#   "Height",
#   "Whole weight",
#   "Shucked weight",
#   "Viscera weight",
#   "Shell weight",
#   "Rings",
# ]

# df.drop('Sex', axis=1, inplace=True)

# df.to_csv('C:\\Users\\TheAncientOwl\\Code\\data-analysis-tool\\server\\common\\test-data\\knn.csv', index=False)

df = pd.read_csv('C:\\Users\\TheAncientOwl\\Code\\data-analysis-tool\\server\\common\\test-data\\knn.csv')
df.head()


,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [10]:
X = df.drop('Rings', axis=1)
X = X.values

y = df['Rings']
y = y.values


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=12345
)


1. Arbitrary K

In [13]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)

knn.fit(X_train, y_train)


KNeighborsRegressor(n_neighbors=3)

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt

train_preds = knn.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
rmse

test_preds = knn.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
rmse


2.375417924000521

2. GridSearchCV for k

In [21]:
from sklearn.model_selection import GridSearchCV

parameters = {
  'n_neighbors': range(1, 50)
}

gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)

gridsearch.fit(X_train, y_train)

gridsearch.best_params_

train_preds_grid = gridsearch.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds_grid)
train_rmse = sqrt(train_mse)
print(train_rmse)

test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)
print(test_rmse)


2.0731180327543384
2.1700197339962175


3. GridSearchCV for k and weights

In [31]:
parameters = {
    "n_neighbors": range(1, 50),
    "weights": ["uniform", "distance"],
}

gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 50),
                         'weights': ['uniform', 'distance']})

print(gridsearch.best_params_)

test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)
test_rmse


{'n_neighbors': 25, 'weights': 'distance'}


2.1634265584947485

4. Bagging and GridSearchCV

In [23]:
best_k = gridsearch.best_params_["n_neighbors"]
best_weights = gridsearch.best_params_["weights"]

bagged_knn = KNeighborsRegressor(
    n_neighbors=best_k, weights=best_weights
)


In [29]:
from sklearn.ensemble import BaggingRegressor

bagging_model = BaggingRegressor(bagged_knn, n_estimators=100)

bagging_model.fit(X_train, y_train)


BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=25,
                                                    weights='distance'),
                 n_estimators=100)

In [30]:
test_preds_grid = bagging_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = sqrt(test_mse)
test_rmse


2.1637830404210705